# gmxapi workshop session (part 2)
## Goal: Simulation-Analysis loop

```python
subgraph = gmx.subgraph(
    variables={
        'found_native': False,
        'checkpoint': '',
        'min_rms': 1e6})
with subgraph:
    md = gmx.mdrun(
        input_list,
        runtime_args={
            '-cpi': subgraph.checkpoint,
            '-maxh': '2',
            '-noappend': None
        })

    subgraph.checkpoint = md.output.checkpoint
    rmsd = gmx.commandline_operation(
        'gmx', ['rms'],
        input_files={
            '-s': reference_struct,
            '-f': md.output.trajectory},
        output_files={'-o': 'rmsd.xvg'},
        stdin='Backbone Backbone\n'
    )
    subgraph.min_rms = numeric_min(
        xvg_to_array(rmsd.output.file['-o']).output.data).output.data
    subgraph.found_native = less_than(lhs=subgraph.min_rms, rhs=0.3).output.data

folding_loop = gmx.while_loop(
    operation=subgraph,
    condition=gmx.logical_not(subgraph.found_native))()
print('Beginning folding_loop.')
folding_loop.run()
print(f'Finished folding_loop. min_rms: {folding_loop.output.min_rms.result()}')
```

## This material

https://github.com/kassonlab/gmxapi-tutorials/tree/csc202202

DOI: [10.1101/2021.07.18.452496v3](https://www.biorxiv.org/content/10.1101/2021.07.18.452496v3)

Funnelweb spider peptide structure files from
* Sorin and Pande, 2005; doi:10.1529/biophysj.104.051938
(used with permission).

## Quick check

In [ ]:
# Assumes we are in `./AdvancedGromacsCourse/gmxapi-tutorials/gmxapi-introduction`
# and the tutorial material is in `../`
# Check!
import os
from pathlib import Path
notebook_dir = Path(os.getcwd())
tutorials_dir = notebook_dir.parent
input_dir = tutorials_dir / 'input_files' / 'fs-peptide'
assert input_dir.exists()

In [ ]:
import gmxapi as gmx
assert gmx.version.api_is_at_least(0, 3) 

In [ ]:
os.makedirs('exercise2', exist_ok=True)
os.chdir('exercise2')

In [ ]:
os.getcwd()

# Helper functions

In [ ]:
import logging
import typing

from gmxapi import function_wrapper

def less_than(lhs: typing.SupportsFloat, rhs: typing.SupportsFloat):
    """Compare the left-hand-side to the right-hand-side.

    Follows the Numpy logic for normalizing the numeric types of *lhs* and *rhs*.
    """
    import numpy as np
    dtype = int
    if any(isinstance(operand, float) for operand in (lhs, rhs)):
        dtype = float
    elif all(isinstance(operand, typing.SupportsFloat) for operand in (lhs, rhs)):
        if type(np.asarray([lhs, rhs])[0].item()) is float:
            dtype = float
    elif any(isinstance(operand, gmxapi.abc.Future) for operand in (lhs, rhs)):
        for item in (lhs, rhs):
            if hasattr(item, 'dtype'):
                if issubclass(item.dtype, (float, np.float)):
                    dtype = float
            elif hasattr(item, 'description'):
                if issubclass(item.description.dtype, (float, np.float)):
                    dtype = float
    else:
        raise UsageError(f'No handling for [{repr(lhs)}, {repr(rhs)}].')

    if dtype is int:
        def _less_than(lhs: int, rhs: int) -> bool:
            return lhs < rhs
    elif dtype is float:
        def _less_than(lhs: float, rhs: float) -> bool:
            return lhs < rhs
    else:
        raise UsageError('Operation only supports standard numeric types.')
    return function_wrapper()(_less_than)(lhs=lhs, rhs=rhs)


@function_wrapper()
def _numpy_min_float(data: gmx.NDArray) -> float:
    import numpy as np
    logging.info(f'Looking for minimum in {data}')
    return float(np.min(data._values))


def numeric_min(array):
    """Find the minimum value in an array.
    """
    return _numpy_min_float(data=array)


@gmx.function_wrapper(output={'data': gmx.NDArray})
def xvg_to_array(path: str, output):
    """Get an NxM array from a GROMACS xvg data file.

    For energy output, N is the number of samples, and the first of M
    columns contains simulation time values.
    """
    import numpy
    logging.info(f'Reading xvg file {path}.')
    data = numpy.genfromtxt(path, comments='@', skip_header=14)
    logging.info(f'Read array shape {data.shape} from {path}.')
    if len(data.shape) == 1:
        # Trajectory was too short. Only a single line was read.
        assert data.shape[0] == 2
        data = data.reshape((1, 2))
    assert len(data.shape) == 2
    output.data = data[:, 1]



## Prepare inputs

### Prepare a molecular model from a PDB file using the `pdb2gmx` GROMACS tool.

In [ ]:
# Confirm inputs exist
assert (input_dir / 'start0.pdb').exists()

In [ ]:
args = ['pdb2gmx', '-ff', 'amber99sb-ildn', '-water', 'tip3p']
input_files = {'-f': os.path.join(input_dir, 'start0.pdb')}
output_files = {
        '-p': 'topol.top',
        '-i': 'posre.itp',
        '-o': 'conf.gro'}
make_top = gmx.commandline_operation('gmx', args, input_files, output_files)

### Prepare the simulation input

Call the GROMACS MD preprocessor to create a simulation input file.

In [ ]:
# Confirm inputs exist.
# assert os.path.exists(make_top.output.file['-o'].result())
# assert os.path.exists(make_top.output.file['-p'].result())
# assert os.path.exists(input_dir / 'grompp.mdp')

In [ ]:
cmd_dir = input_dir

grompp_input_files = {'-f': os.path.join(cmd_dir, 'grompp.mdp'),
                      '-c': make_top.output.file['-o'],
                      '-p': make_top.output.file['-p']}

grompp = gmx.commandline_operation(
    'gmx', ['grompp'],
    input_files = [grompp_input_files],
    output_files = {'-o': 'run.tpr'})
tpr_input = grompp.output.file['-o'].result()

In [ ]:
input_list = gmx.read_tpr(tpr_input)

#### Inspect

In [ ]:
input_list.output.parameters.result()

#### Adjust input parameters

In [ ]:
input_list = gmx.modify_input(input_list, parameters={'nstxout': 100})

## Looping and custom operations

In [ ]:
reference_struct = input_dir / 'ref.pdb'
assert reference_struct.exists()

In [ ]:
ensemble_size = 1
allocated_cores = 4

In [ ]:
subgraph = gmx.subgraph(
    variables={
        'found_native': False,
        'checkpoint': '',
        'min_rms': 1e6})
with subgraph:
    md = gmx.mdrun(
        input_list,
        runtime_args={
            '-cpi': subgraph.checkpoint,
            '-maxh': '0.001',
            '-noappend': None,
            '-nt': str(allocated_cores // ensemble_size)
        })

    subgraph.checkpoint = md.output.checkpoint
    rmsd = gmx.commandline_operation(
        'gmx', ['rms'],
        input_files={
            '-s': reference_struct,
            '-f': md.output.trajectory},
        output_files={'-o': 'rmsd.xvg'},
        stdin='Backbone Backbone\n'
    )
    subgraph.min_rms = numeric_min(
        xvg_to_array(rmsd.output.file['-o']).output.data).output.data
    subgraph.found_native = less_than(lhs=subgraph.min_rms, rhs=0.3).output.data

folding_loop = gmx.while_loop(
    operation=subgraph,
    condition=gmx.logical_not(subgraph.found_native))()
print('Beginning folding_loop.')
folding_loop.run()
print(f'Finished folding_loop. min_rms: {folding_loop.output.min_rms.result()}')

# Exercise 3: Ensemble simulation
Use the built-in mpi4py ensemble executor.

In [ ]:
while not str(os.getcwd()).endswith('gmxapi-introduction'):
    os.chdir('..')
os.getcwd()

In [ ]:
os.makedirs('exercise3', exist_ok=True)
os.chdir('exercise3')

In [ ]:
script_dir = input_dir.parent.parent / 'examples'
example = script_dir / 'fs-peptide.py'
assert os.path.exists(example)

In [ ]:
!g=($(printenv | grep SLURM | cut -d "=" -f1 )); for a in "${g[@]}" ; do unset $a ; done; mpirun  -n 4 -hosts=localhost python3 -m mpi4py -c 'import gmxapi; from mpi4py import MPI; print(MPI.COMM_WORLD.Get_rank())'

In [ ]:
!echo mpiexec -n 5 `which python` -m mpi4py $example

In [ ]:
!srun --help
